In [132]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import random


In [133]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])
a = open('processed.cleveland.data', 'r')
c = a.readlines()
random.shuffle(c) #shuffle data before using
n_val = []
n_keys = []
d = []
for elem in c:
    elem = elem[:-1]
    elem = list(map(float,elem.split(',')))
    n_val.append(elem[:-1])
    n_keys.append(elem[-1]) #mb ad
rot = [[] for i in range(len(n_val[0]))]
mean = []
std = []
for i in range(len(n_val)):
    for j in range(len(n_val[i])):
        rot[j].append(n_val[i][j])
#print(len(rot[0]))
for elem in rot:
    #print(elem)
    new_elem = np.array(elem)
    #print(np.std(new_elem))
    #print(new_elem.shape)
    std.append(np.std(new_elem))
    mean.append(np.mean(new_elem))
    #elem = [int(elem[i]*10) for i in range(len(elem))] 
for i in range(len(rot)):
    rot[i] = [(rot[i][j] - mean[i])/std[i] for j in range(len(elem))]
#print(rot[0])
rot = np.array(rot)
n_val = rot.T     #all of the previous 20 lines normalises our array


#true-false
for i in range(len(n_keys)):
    if n_keys[i] > 0:
        n_keys[i] = 1
print(n_keys)

[0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 1, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 1, 1, 1, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 1, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 0.0, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 1, 1, 1, 0.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 0.0, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 1, 1, 1, 1, 0.0, 1, 

In [134]:
#73 - test size
#230 - train size
t = 230
te = 73
a = random.sample(range(303), t)
val, keys, te_val, te_keys = [],[],[],[]
for i in range(len(n_val)):
    if i in a:
        val.append(n_val[i])
        keys.append(n_keys[i])
    else:
        te_val.append(n_val[i])                       #train-test 
        te_keys.append(n_keys[i])

val = np.array(val)
keys = np.array(keys)
#val = val.astype(float)
#keys = keys.astype(float)

te_val = np.array(te_val)
te_keys = np.array(te_keys)
#keys.shape
val.shape

(225, 13)

In [135]:
input_size = 13
hidden_sizes = [128, 128]
output_size = 2

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model) #model with 3 layers, first 2 with relu, last with logistic curve

Sequential(
  (0): Linear(in_features=13, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=2, bias=True)
  (5): LogSoftmax()
)


In [136]:
criterion = nn.NLLLoss() #MSELoss() 
#images, labels = next(iter(trainloader))
#images = images.view(images.shape[0], -1)

#logps = model(images) #log probabilities
#loss = criterion(logps, labels) #calculate the NLL loss

In [137]:
keys[0]

0.0

In [138]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
time0 = time()
epochs = 30
for e in range(epochs):
    running_loss = 0
    for i in range(len(val)):
        # Training pass
        optimizer.zero_grad()
        temp_1 = [val[i]]
        temp0 = np.array(temp_1)
        temp = torch.from_numpy(temp0)
        temp1 = temp.float()
        #print(temp1)
        output = model(temp1)    #DOES NOT WORK. MODEL RETURNS NAN TENSOR
        #print(output)
        k = [keys[i]]
        k = np.array(k)
        k = torch.from_numpy(k)
        k = k.long()
        #print(output.shape)
        #print(k.shape)
        #print('result', e, output)
        #print(k)
        loss = criterion(output, k)
        #print(loss)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        #print(loss.item())
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(val)))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.47246257845105394
Epoch 1 - Training loss: 0.36159670289838686
Epoch 2 - Training loss: 0.3086192248691481
Epoch 3 - Training loss: 0.2644419637817878
Epoch 4 - Training loss: 0.22310542189092203
Epoch 5 - Training loss: 0.18992356225372015
Epoch 6 - Training loss: 0.1517888315872963
Epoch 7 - Training loss: 0.15815258738369375
Epoch 8 - Training loss: 0.10645577385676303
Epoch 9 - Training loss: 0.0811360431769501
Epoch 10 - Training loss: 0.07258982644413815
Epoch 11 - Training loss: 0.0596769250681537
Epoch 12 - Training loss: 0.043876746637515146
Epoch 13 - Training loss: 0.05491245090027721
Epoch 14 - Training loss: 0.30292342605293004
Epoch 15 - Training loss: 0.14503666615039384
Epoch 16 - Training loss: 0.08022974246899375
Epoch 17 - Training loss: 0.03043984462018039
Epoch 18 - Training loss: 0.013861719567016262
Epoch 19 - Training loss: 0.007235605643078568
Epoch 20 - Training loss: 0.004534909595122308
Epoch 21 - Training loss: 0.00348711704846402

In [139]:
correct_count, all_count = 0, 0    
norm_c, norm_true = 0,0
dis_c, dis_true = 0,0

for i in range(len(te_val)):
    temp_1 = [te_val[i]]
    temp0 = np.array(temp_1)
    temp = torch.from_numpy(temp0)
    temp1 = temp.float()
    with torch.no_grad():
        logps = model(temp1)
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = te_keys[i]
    if(true_label == pred_label):
        correct_count += 1
        if true_label == 0:
            norm_true += 1
        else:
            dis_true += 1
    if true_label == 0:
        norm_c += 1
    else:
        dis_c += 1        
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 72

Model Accuracy = 0.7361111111111112


In [140]:
#true-false 80 percent
print("Number Of norm =", norm_c)
print("\nModel Accuracy norm=", (norm_true/norm_c))
print("Number Of dis =", dis_c)
print("\nModel Accuracy dis=", (dis_true/dis_c))

Number Of norm = 38

Model Accuracy norm= 0.8157894736842105
Number Of dis = 34

Model Accuracy dis= 0.6470588235294118


In [ ]:
torch.save(model, './my_mnist_model.pt') 